<a href="https://colab.research.google.com/github/pinkesh-ai/DL201/blob/main/10_PPAi21V2_nlp_own_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Deep Dive - Own Code

Own refactored code and notes for *Chapter 10: NLP Deep Dive: RNNs* ([`10_nlp.ipynb`](https://colab.research.google.com/github/vtecftwy/fastbook/blob/master/10_nlp.ipynb)).

## Instructions

It is recommended that you work in two steps:
1. Copy the code from the fastbook notebook and make sure it works
2. Refactor (i.e. rewrite the code in your own style) by 
    - regrouping things together that make sense ro you
    - adding text cells to explain what to code does in your own words and possible references to the doc you may have consulsted
    - deleting code you think was only there to explain things but are not required once you run models end to end

When you have done that, you get a customized reference notebook for you which you can consult later on when you forgot the details, withouht having to read the full notebook from fastbook.

10: NLP - Patel 

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 8.0MB/s 
     |████████████████████████████████| 194kB 37.7MB/s 
     |████████████████████████████████| 51kB 9.7MB/s 
     |████████████████████████████████| 1.2MB 50.7MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
     |████████████████████████████████| 61kB 11.3MB/s 
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

In [3]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [4]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [5]:
txt = files[0].open().read(); txt[:75]

'Actually, I lied: Paul Newman deserved the Oscar for "The Verdict". Anyway,'

In [6]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#155) ['Actually',',','I','lied',':','Paul','Newman','deserved','the','Oscar','for','"','The','Verdict','"','.','Anyway',',','Peter',"O'Toole",'gives','another','one','of','his','superlative','performances','.','How','this'...]


In [7]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [8]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#184) ['xxbos','xxmaj','actually',',','i','lied',':','xxmaj','paul','xxmaj','newman','deserved','the','xxmaj','oscar','for','"','the','xxmaj','verdict','"','.','xxmaj','anyway',',','xxmaj','peter',"o'toole",'gives','another','one'...]


In [9]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#184) ['xxbos','xxmaj','actually',',','i','lied',':','xxmaj','paul','xxmaj','newman','deserved','the','xxmaj','oscar','for','"','the','xxmaj','verdict','"','.','xxmaj','anyway',',','xxmaj','peter',"o'toole",'gives','another','one'...]


Default Rules 

In [10]:
txts = L(o.open().read() for o in files[:2000])

In [11]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [12]:
subword(1000)

'▁A c t u ally , ▁I ▁li ed : ▁P a ul ▁New man ▁deserve d ▁the ▁O s c ar ▁for ▁" The ▁V er d ic t " . ▁An y way , ▁P e ter ▁O'

In [13]:
subword(200)

'▁A c t u al ly , ▁I ▁ li ed : ▁ P a u l ▁ N e w m an ▁d es er ve d ▁the ▁ O s c ar ▁for ▁ " T h e'

In [14]:
subword(10000)

'▁Actually , ▁I ▁li ed : ▁Paul ▁Newman ▁deserved ▁the ▁Oscar ▁for ▁" The ▁Ver d ic t ". ▁Anyway , ▁Peter ▁O \' Toole ▁give s ▁another ▁one ▁of ▁his ▁superlative ▁performances . ▁How ▁this ▁man ▁doesn \' t'

In [15]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#184) ['xxbos','xxmaj','actually',',','i','lied',':','xxmaj','paul','xxmaj','newman','deserved','the','xxmaj','oscar','for','"','the','xxmaj','verdict','"','.','xxmaj','anyway',',','xxmaj','peter',"o'toole",'gives','another','one'...]


In [16]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#184) ['xxbos','xxmaj','actually',',','i','lied',':','xxmaj','paul','xxmaj'...]

In [17]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#1960) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','it','in','i'...]"

In [18]:
nums = num(toks)[:20]; nums

TensorText([   2,    8,  224,   11,   19,    0,  111,    8,  563,    8,    0, 1435,    9,    8,  461,   27,   20,    9,    8,    0])

In [19]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj actually , i xxunk : xxmaj paul xxmaj xxunk deserved the xxmaj oscar for " the xxmaj xxunk'

Batches =No Resize 

In [20]:
nums200 = toks200.map(num)

In [21]:
dl = LMDataLoader(nums200)

In [22]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [23]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj actually , i xxunk : xxmaj paul xxmaj xxunk deserved the xxmaj oscar for " the xxmaj xxunk'

In [24]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj actually , i xxunk : xxmaj paul xxmaj xxunk deserved the xxmaj oscar for " the xxmaj xxunk "'

In [25]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [26]:
dls_lm.show_batch(max_n=2)

,text,text_
0,xxbos xxmaj xxunk ( which reads xxmaj starstruck on my box ) was the most amazing movie i have ever seen . i thought that it was one of the best movies i have ever seen . xxmaj so why not a 10 ? xxmaj nothing is perfect . xxmaj jamie xxmaj kennedy proves why he is one of my favorite actors in this very interesting look at a darker side of xxmaj hollywood . i have forced a few,xxmaj xxunk ( which reads xxmaj starstruck on my box ) was the most amazing movie i have ever seen . i thought that it was one of the best movies i have ever seen . xxmaj so why not a 10 ? xxmaj nothing is perfect . xxmaj jamie xxmaj kennedy proves why he is one of my favorite actors in this very interesting look at a darker side of xxmaj hollywood . i have forced a few others
1,"can repel you for several pages of story and take your mind away from what 's happening on the current page . xxmaj the sheriff in xxmaj texas picked up the phone and called xxmaj san xxmaj francisco based on a badly drawn composite . "" yeah , we 've got your guy . i was picking my ass looking for something to do and i noticed the striking similarity between one of our rape suspects and your stick -","repel you for several pages of story and take your mind away from what 's happening on the current page . xxmaj the sheriff in xxmaj texas picked up the phone and called xxmaj san xxmaj francisco based on a badly drawn composite . "" yeah , we 've got your guy . i was picking my ass looking for something to do and i noticed the striking similarity between one of our rape suspects and your stick - man"


In [27]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [28]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.013940,3.903325,0.299751,49.567001,21:09


In [29]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [30]:
learn = learn.load('1epoch')

In [31]:
learn.unfreeze()
learn.fit_one_cycle(1, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.675709,3.675731,0.326583,39.477512,22:43


In [32]:
learn.save_encoder('finetuned')

In [34]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [36]:
print("\n".join(preds))

i liked this movie because of the charming characters , the likeable Joey and the tough - guy Joey . 

 The movie is not about the Mafia but about the power of the people . It 's a great
i liked this movie because it was made for television . It 's a story of two brothers , both played by Dennis Hopper and John Travolta . However , they never really know each other . Being


In [37]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [38]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj by now you 've probably heard a bit about the new xxmaj disney dub of xxmaj miyazaki 's classic film , xxmaj laputa : xxmaj castle xxmaj in xxmaj the xxmaj sky . xxmaj during late summer of 1998 , xxmaj disney released "" kiki 's xxmaj delivery xxmaj service "" on video which included a preview of the xxmaj laputa dub saying it was due out in "" 1 xxrep 3 9 "" . xxmaj it 's obviously way past that year now , but the dub has been finally completed . xxmaj and it 's not "" laputa : xxmaj castle xxmaj in xxmaj the xxmaj sky "" , just "" castle xxmaj in xxmaj the xxmaj sky "" for the dub , since xxmaj laputa is not such a nice word in xxmaj spanish ( even though they use the word xxmaj laputa many times",pos
2,"xxbos xxmaj warning : xxmaj does contain spoilers . \n\n xxmaj open xxmaj your xxmaj eyes \n\n xxmaj if you have not seen this film and plan on doing so , just stop reading here and take my word for it . xxmaj you have to see this film . i have seen it four times so far and i still have n't made up my mind as to what exactly happened in the film . xxmaj that is all i am going to say because if you have not seen this film , then stop reading right now . \n\n xxmaj if you are still reading then i am going to pose some questions to you and maybe if anyone has any answers you can email me and let me know what you think . \n\n i remember my xxmaj grade 11 xxmaj english teacher quite well . xxmaj",pos


In [39]:
nums_samp = toks200[:10].map(num)

In [40]:
nums_samp.map(len)

(#10) [184,136,248,91,179,161,327,273,200,207]

In [41]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [42]:
learn = learn.load_encoder('finetuned')

In [43]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.286695,0.230190,0.907920,01:11


In [44]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.244555,0.190130,0.926400,01:16


In [45]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.204420,0.170254,0.934680,01:36


In [46]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.162198,0.173507,0.935080,01:57
1,0.148791,0.171044,0.936720,01:57


In [ ]:
pretrained model+ fine-tuned it for task+ body (the encoder)+ New Head( Encoder) 